In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup as BS
import requests
import time as t
from selenium import webdriver

In [2]:
yoox=pd.read_csv('yoox.csv').drop('Unnamed: 0',axis=1)

In [3]:
yoox.head()

,price,category,brand,size,url
0,"<div class=""price"">\n<span class=""fullprice fo...","<div class=""microcategory font-sans"">\r\n ...","<div class=""brand font-bold text-uppercase"">8 ...","<div class=""size text-light"">\n<span class=""aS...","<a class=""itemlink"" href=""/fr/41925191HP/item#..."
1,"<div class=""price"">\n<span class=""fullprice fo...","<div class=""microcategory font-sans"">\r\n ...","<div class=""brand font-bold text-uppercase"">CH...","<div class=""size text-light"">\n<span class=""aS...","<a class=""itemlink"" href=""/fr/42761341PK/item#..."
2,"<div class=""price"">\n<span class=""fullprice fo...","<div class=""microcategory font-sans"">\r\n ...","<div class=""brand font-bold text-uppercase"">8 ...","<div class=""size text-light"">\n<span class=""aS...","<a class=""itemlink"" href=""/fr/35421679UB/item#..."
3,"<div class=""price"">\n<span class=""oldprice tex...","<div class=""microcategory font-sans"">\r\n ...","<div class=""brand font-bold text-uppercase"">MA...","<div class=""size text-light"">\n<span class=""aS...","<a class=""itemlink"" href=""/fr/14007863DH/item#..."
4,"<div class=""price"">\n<span class=""fullprice fo...","<div class=""microcategory font-sans"">\r\n ...","<div class=""brand font-bold text-uppercase"">MA...","<div class=""size text-light"">\n<span class=""aS...","<a class=""itemlink"" href=""/fr/35421773RV/item#..."


In [4]:
yoox.price[3]

'<div class="price">\n<span class="oldprice text-linethrough text-light">EUR 284,00</span>\n<span class="newprice font-bold">EUR 218,00</span>\n</div>'

In [5]:
yoox.price=yoox.price.str.replace('<div class="price">\n<span class="fullprice font-bold">EUR ','').str.replace('<div class="price">\n<span class="oldprice text-linethrough text-light">EUR ','').str.replace('</span>\n<span class="newprice font-bold">',' - ').str.replace('</span>\n</div>','').str.replace("EUR ",'').str.replace('.','').str.replace(",",'.').str.split('-')

In [6]:
yoox.price.head()

0              [249.00]
1              [511.00]
2               [99.00]
3    [284.00 ,  218.00]
4              [421.00]
Name: price, dtype: object

In [7]:
yoox['price_original']=yoox.price.apply(lambda x: x[0]).str.strip(' ').str.join('')

In [8]:
yoox.price_original.head()

0    249.00
1    511.00
2     99.00
3    284.00
4    421.00
Name: price_original, dtype: object

In [9]:
yoox['price_actual']=yoox.price.apply(lambda x: x[1] if len(x)==2 else x[0]).str.strip(' ').str.join('')

In [10]:
yoox.price_actual.head()

0    249.00
1    511.00
2     99.00
3    218.00
4    421.00
Name: price_actual, dtype: object

In [11]:
yoox.drop('price',axis=1,inplace=True)

In [12]:
yoox.category[0]

'<div class="microcategory font-sans">\r\n                                Manteau long en velours\r\n                            </div>'

In [13]:
yoox.category=yoox.category.str.replace('<div class="microcategory font-sans">\r\n','').str.replace('\r\n','').str.replace('</div>','').str.strip(' ')

In [14]:
yoox.brand[0]

'<div class="brand font-bold text-uppercase">8 by YOOX</div>'

In [15]:
yoox.brand=yoox.brand.str.replace('</div>','').str.replace('<div class="brand font-bold text-uppercase">','')

In [16]:
yoox['size'][0]

'<div class="size text-light">\n<span class="aSize">34</span>\n<span class="aSize">36</span>\n<span class="aSize">38</span>\n<span class="aSize">40</span>\n<span class="aSize">42</span>\n</div>'

In [17]:
yoox['size']=yoox['size'].str.replace('<div class="size text-light">\n<span class="aSize">','').str.replace('</span>\n</div>','').str.replace('</span>\n<span class="aSize">',',')

In [18]:
yoox['size']=yoox['size'].str.replace("XXXS","30").str.replace("XXS",'32').str.replace('XS','34').str.replace('S','36').str.replace('XXXXL','48').str.replace('XXXL','46').str.replace('XXL','44').str.replace('XL','42').str.replace('L','40').str.replace('M','38').str.replace("2940",'29L').str.replace("3040",'29L').str.replace("3140",'29L').str.replace("3240",'29L').str.replace("3440",'29L').str.replace("3540",'29L').str.replace("3640",'29L').str.replace("ONE36IZE",'ONESIZE')

In [19]:
yoox['size'].value_counts()

36                                                                   17918
38                                                                   17844
40                                                                   11231
34,36,38,40                                                           8456
36,38                                                                 8348
36,38,40                                                              7869
34                                                                    7142
38,40                                                                 5735
34,36,38                                                              5442
42                                                                    5320
34,36,38,40,42                                                        5163
34,36                                                                 4269
36,38,40,42                                                           3774
36,40                    

In [20]:
yoox.isna().sum()

category            0
brand               0
size              188
url                 0
price_original      0
price_actual        0
dtype: int64

In [21]:
yoox.dropna(inplace=True)

In [22]:
yoox.drop(yoox[(yoox['size'].str.contains(r'[A-z]'))].index, inplace=True)

In [23]:
yoox.drop(yoox[(yoox['size'].str.contains(r'[\d]{3,}'))].index, inplace=True)

In [24]:
yoox.drop(yoox[(yoox['size'].str.contains(r'(8\d)'))].index, inplace=True)

In [25]:
yoox.drop(yoox[(yoox['size'].str.contains(r'(2\d)'))].index, inplace=True)

In [26]:
yoox.drop(yoox[(yoox['size'].str.contains('/'))].index, inplace=True)

In [27]:
yoox.drop(yoox[(yoox['size'].str.contains('-'))].index, inplace=True)

In [28]:
yoox.drop(yoox[yoox['size']=='1'].index,inplace=True)

In [29]:
yoox.shape

(146774, 6)

In [30]:
yoox.head()

,category,brand,size,url,price_original,price_actual
0,Manteau long en velours,8 by YOOX,"34,36,38,40,42","<a class=""itemlink"" href=""/fr/41925191HP/item#...",249.00,249.00
1,Chemises en jean,CHLOÉ,"34,36,38","<a class=""itemlink"" href=""/fr/42761341PK/item#...",511.00,511.00
2,Jupe mi-longue à motif écossais,8 by YOOX,"34,36,38,40,42","<a class=""itemlink"" href=""/fr/35421679UB/item#...",99.00,99.00
3,Pullovers,MARNI,36,"<a class=""itemlink"" href=""/fr/14007863DH/item#...",284.00,218.00
4,Jupes mi-longues,MARNI,"34,36,38,42","<a class=""itemlink"" href=""/fr/35421773RV/item#...",421.00,421.00


In [31]:
yoox.reset_index(drop=True,inplace=True)

In [32]:
yoox['size']=yoox['size'].str.replace("'",'')

In [33]:
size_list=['30','32','34','36','38','40','42','44','46','48']
yoox['size']=yoox['size'].str.split(',')

In [34]:
yoox['size_check']=yoox['size'].apply(lambda x: min([i in size_list for i in x]))

In [35]:
yoox.size_check.value_counts()

True     142909
False      3865
Name: size_check, dtype: int64

In [36]:
yoox.drop(yoox[yoox['size_check']==False].index,inplace=True)

In [37]:
yoox.drop('size_check',axis=1,inplace=True)

In [38]:
yoox['size_30']=yoox['size'].apply(lambda x: str([1 for i in x if i=='30']).strip('[]') or 0 ).astype(int)
yoox['size_32']=yoox['size'].apply(lambda x: str([1 for i in x if i=='32']).strip('[]') or 0 ).astype(int)
yoox['size_34']=yoox['size'].apply(lambda x: str([1 for i in x if i=='34']).strip('[]') or 0 ).astype(int)
yoox['size_36']=yoox['size'].apply(lambda x: str([1 for i in x if i=='36']).strip('[]') or 0 ).astype(int)
yoox['size_38']=yoox['size'].apply(lambda x: str([1 for i in x if i=='38']).strip('[]') or 0 ).astype(int)
yoox['size_40']=yoox['size'].apply(lambda x: str([1 for i in x if i=='40']).strip('[]') or 0 ).astype(int)
yoox['size_42']=yoox['size'].apply(lambda x: str([1 for i in x if i=='42']).strip('[]') or 0 ).astype(int)
yoox['size_44']=yoox['size'].apply(lambda x: str([1 for i in x if i=='44']).strip('[]') or 0 ).astype(int)
yoox['size_46']=yoox['size'].apply(lambda x: str([1 for i in x if i=='46']).strip('[]') or 0 ).astype(int)
yoox['size_48']=yoox['size'].apply(lambda x: str([1 for i in x if i=='48']).strip('[]') or 0 ).astype(int)

In [39]:
yoox.drop('size',axis=1,inplace=True)

In [40]:
yoox.columns

Index(['category', 'brand', 'url', 'price_original', 'price_actual', 'size_30',
       'size_32', 'size_34', 'size_36', 'size_38', 'size_40', 'size_42',
       'size_44', 'size_46', 'size_48'],
      dtype='object')

In [41]:
yoox.dtypes

category          object
brand             object
url               object
price_original    object
price_actual      object
size_30            int32
size_32            int32
size_34            int32
size_36            int32
size_38            int32
size_40            int32
size_42            int32
size_44            int32
size_46            int32
size_48            int32
dtype: object

In [42]:
yoox['size_32'].value_counts()

0    137030
1      5879
Name: size_32, dtype: int64

In [43]:
yoox.price_original[yoox.price_original.str.contains(r'[A-z]')].value_counts()

<div class="price">\n<div>ARTICLE ÉPUISÉ</div>\n</div>    167
Name: price_original, dtype: int64

In [44]:
yoox.drop(yoox[yoox.price_original.str.contains(r'[A-z]')].index,inplace=True)

In [45]:
yoox[['price_original', 'price_actual']]=yoox[['price_original', 'price_actual']].astype('float')

In [46]:
yoox.dtypes

category           object
brand              object
url                object
price_original    float64
price_actual      float64
size_30             int32
size_32             int32
size_34             int32
size_36             int32
size_38             int32
size_40             int32
size_42             int32
size_44             int32
size_46             int32
size_48             int32
dtype: object

In [47]:
yoox['discount_rate']=round(1-(yoox.price_actual/yoox.price_original),2)

In [48]:
yoox['discount_rate'].value_counts().head()

0.00    74017
0.50     2837
0.60     2644
0.56     2237
0.58     1987
Name: discount_rate, dtype: int64

In [49]:
yoox['new_category']='0'

In [50]:
yoox.category[yoox.new_category.str.contains('0')].value_counts()

Pantalons                                                   21743
Robes courtes                                               11895
Pantalons en jean                                            9777
Blouses                                                      8617
Pullovers                                                    7898
T-shirts                                                     6834
Tops                                                         6265
Vestes                                                       5658
Modèle fourreau                                              4392
Cardigans                                                    3514
Robes longues                                                3441
Robes aux genoux                                             3424
Jupes au genou                                               3168
Sweat-shirts                                                 3051
Jupes mi-longues                                             2884
Chemises e

In [51]:
yoox['new_category'][yoox.category.str.contains('Pantalon')]='Trousers'
yoox['new_category'][yoox.category.str.contains('Robes courtes')]='Mini Dress'
yoox['new_category'][yoox.category.str.contains('Robes mi-longues')]='Midi Dress'
yoox['new_category'][yoox.category.str.contains('Robes au genou')| yoox.category.str.contains('Robes aux genoux')]='Knee Length Dress'
yoox['new_category'][yoox.category.str.contains('Robes longues')|yoox.category.str.contains('Robe longue')]='Maxi Dress'
yoox['new_category'][yoox.category.str.contains('Modèle fourreau')]='Bodycon Dress'
yoox['new_category'][yoox.category.str.contains('Jupes courtes')|yoox.category.str.contains('Mini-jupes')]='Mini Skirt'
yoox['new_category'][yoox.category.str.contains('Jupes mi-longues')]='Midi Skirt'
yoox['new_category'][yoox.category.str.contains('Jupes au genou')| yoox.category.str.contains('Robes aux genoux')]='Knee Length Skirt'
yoox['new_category'][yoox.category.str.contains('Jupes longues')]='Maxi Skirt'
yoox['new_category'][yoox.category.str.contains('Pantalons en jean')]='Denim Pants'
yoox['new_category'][yoox.category.str.contains('Blouse')|yoox.category.str.contains('Modèle chemisier')|yoox.category.str.contains('Chemisier')]='Blouses'
yoox['new_category'][yoox.category.str.contains('Pull')]='Jumpers'
yoox['new_category'][yoox.category.str.contains('T-shirt')]='T-shirts'
yoox['new_category'][yoox.category.str.contains('Tops')]='Tops'
yoox['new_category'][yoox.category.str.contains('Veste')]='Jackets'
yoox['new_category'][yoox.category.str.contains('Cardigan')]='Cardigans'
yoox['new_category'][yoox.category.str.contains('Sweat-shirts')]='Hoodies'
yoox['new_category'][yoox.category.str.contains('Chemise')]='Shirts'
yoox['new_category'][yoox.category.str.contains('Combinaison')]='Jumpsuits'
yoox['new_category'][yoox.category.str.contains('Cardigan')]='Cardigans'
yoox['new_category'][yoox.category.str.contains('Maillots deux pièces')|yoox.category.str.contains('Bikini')]='Bikini'
yoox['new_category'][yoox.category.str.contains('Caban et veste croisée')|yoox.category.str.contains('Manteau')]='Coats'
yoox['new_category'][yoox.category.str.contains('Manteaux longs')]='Long Coats'
yoox['new_category'][yoox.category.str.contains('Pardessus')|yoox.category.str.contains('Trench')]='Trench Coats'
yoox['new_category'][yoox.category.str.contains('Jupes-culottes')]='Culottes'
yoox['new_category'][yoox.category.str.contains('Cols roulés')]='Turtle Necks'
yoox['new_category'][yoox.category.str.contains('Blouson')|yoox.category.str.contains('Bomber')]='Bomber Jackets'
yoox['new_category'][yoox.category.str.contains('Pyjama')]='Pyjamas'
yoox['new_category'][yoox.category.str.contains('Short')]='Shorts'
yoox['new_category'][yoox.category.str.contains('Maillots une pièce')]='One piece Swimsuit'
yoox['new_category'][yoox.category.str.contains('Doudoune')]='Down-coat'


In [52]:
yoox['new_category'].value_counts()

Trousers              22094
Mini Dress            11895
Blouses               10387
Denim Pants            9777
Jumpers                8652
0                      6876
T-shirts               6847
Tops                   6839
Shirts                 6611
Knee Length Skirt      6592
Jackets                5695
Bodycon Dress          4392
Bomber Jackets         4177
Cardigans              3518
Maxi Dress             3453
Hoodies                3051
Midi Skirt             2884
Trench Coats           2023
Jumpsuits              1969
Midi Dress             1924
Bikini                 1780
Shorts                 1772
Long Coats             1654
Culottes               1451
Turtle Necks           1409
Maxi Skirt             1401
Mini Skirt             1368
Coats                   709
Down-coat               587
One piece Swimsuit      485
Pyjamas                 470
Name: new_category, dtype: int64

In [53]:
yoox.drop(yoox[yoox.new_category.str.contains('0')].index,inplace=True)

In [54]:
yoox.shape

(135866, 17)

In [55]:
yoox.drop(['category','price_original'],axis=1,inplace=True)

In [56]:
yoox.columns=['brand', 'url', 'price_actual', 'size_30', 'size_32',
       'size_34', 'size_36', 'size_38', 'size_40', 'size_42', 'size_44',
       'size_46', 'size_48', 'discount_rate', '']

In [57]:
yoox=pd.get_dummies(data=yoox,columns=[''],drop_first=True)

In [58]:
yoox.head()

,brand,url,price_actual,size_30,size_32,size_34,size_36,size_38,size_40,size_42,...,_Mini Skirt,_One piece Swimsuit,_Pyjamas,_Shirts,_Shorts,_T-shirts,_Tops,_Trench Coats,_Trousers,_Turtle Necks
0,8 by YOOX,"<a class=""itemlink"" href=""/fr/41925191HP/item#...",249.0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,CHLOÉ,"<a class=""itemlink"" href=""/fr/42761341PK/item#...",511.0,0,0,1,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,MARNI,"<a class=""itemlink"" href=""/fr/14007863DH/item#...",218.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MARNI,"<a class=""itemlink"" href=""/fr/35421773RV/item#...",421.0,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,8 by YOOX,"<a class=""itemlink"" href=""/fr/13381785CM/item#...",119.0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1,0


In [59]:
yoox.url[0]

'<a class="itemlink" href="/fr/41925191HP/item#dept=clothingwomen&amp;sts=SearchResult">\n<div class="brand font-bold text-uppercase">8 by YOOX</div>\n<div class="microcategory font-sans">\r\n                                Manteau long en velours\r\n                            </div>\n<div class="price">\n<span class="fullprice font-bold">EUR 249,00</span>\n</div>\n<div class="colorSize">\n<div class="size text-light">\n<span class="aSize">34</span>\n<span class="aSize">36</span>\n<span class="aSize">38</span>\n<span class="aSize">40</span>\n<span class="aSize">42</span>\n</div>\n</div>\n</a>'

In [60]:
yoox.url=yoox.url.str.extract(r'href="(.*item?)#').apply(lambda x: 'http://yoox.com'+x)

In [61]:
yoox.head()

,brand,url,price_actual,size_30,size_32,size_34,size_36,size_38,size_40,size_42,...,_Mini Skirt,_One piece Swimsuit,_Pyjamas,_Shirts,_Shorts,_T-shirts,_Tops,_Trench Coats,_Trousers,_Turtle Necks
0,8 by YOOX,http://yoox.com/fr/41925191HP/item,249.0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,CHLOÉ,http://yoox.com/fr/42761341PK/item,511.0,0,0,1,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,MARNI,http://yoox.com/fr/14007863DH/item,218.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MARNI,http://yoox.com/fr/35421773RV/item,421.0,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,8 by YOOX,http://yoox.com/fr/13381785CM/item,119.0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1,0


In [62]:
yoox.columns

Index(['brand', 'url', 'price_actual', 'size_30', 'size_32', 'size_34',
       'size_36', 'size_38', 'size_40', 'size_42', 'size_44', 'size_46',
       'size_48', 'discount_rate', '_Blouses', '_Bodycon Dress',
       '_Bomber Jackets', '_Cardigans', '_Coats', '_Culottes', '_Denim Pants',
       '_Down-coat', '_Hoodies', '_Jackets', '_Jumpers', '_Jumpsuits',
       '_Knee Length Skirt', '_Long Coats', '_Maxi Dress', '_Maxi Skirt',
       '_Midi Dress', '_Midi Skirt', '_Mini Dress', '_Mini Skirt',
       '_One piece Swimsuit', '_Pyjamas', '_Shirts', '_Shorts', '_T-shirts',
       '_Tops', '_Trench Coats', '_Trousers', '_Turtle Necks'],
      dtype='object')

In [63]:
yoox['item']=yoox.url.str.strip('htp:yox.cm/fr/item')

In [64]:
yoox['colors']=0

In [65]:
yoox['compositions']=0

In [78]:
yoox.reset_index(drop=True,inplace=True)

In [79]:
for i in range(50):
    if i%5==0:
        t.sleep(3)
    yoox.colors[i]=str(BS(requests.get(f'http://yoox.com/fr/{yoox.item[i]}/item').content).select(f'#color{yoox.item[i]} > img'))
    if i%5==0:
        print(i)

0
5
10
15
20
25
30
35
40
45


In [80]:
yoox.colors.value_counts()

0                                                                                                                                                                        135816
[<img alt="Sable" class="hide" src="https://www.yoox.com/images/items/38/38860533fh_14_f.jpg?width=38&amp;height=36&amp;impolicy=crop&amp;gravity=Center"/>]                  1
[<img alt="Camel" class="hide" src="https://www.yoox.com/images/items/39/39936323uf_14_f.jpg?width=38&amp;height=36&amp;impolicy=crop&amp;gravity=Center"/>]                  1
[<img alt="Bleu d'azur" class="hide" src="https://www.yoox.com/images/items/13/13299265bq_14_f.jpg?width=38&amp;height=36&amp;impolicy=crop&amp;gravity=Center"/>]            1
[<img alt="Camel" class="hide" src="https://www.yoox.com/images/items/34/34867585nr_14_f.jpg?width=38&amp;height=36&amp;impolicy=crop&amp;gravity=Center"/>]                  1
[<img alt="Beige" class="hide" src="https://www.yoox.com/images/items/39/39989632td_14_f.jpg?width=38&amp;height=36&amp;

In [82]:
for i in range(50,500):
    if i%5==0:
        t.sleep(3)
    yoox.colors[i]=str(BS(requests.get(f'http://yoox.com/fr/{yoox.item[i]}/item').content).select(f'#color{yoox.item[i]} > img'))
    if i%20==0:
        print(i)

60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480


In [83]:
for i in range(500):
    if i%5==0:
        t.sleep(3)
    yoox.compositions[i]=str(BS(requests.get(f'http://yoox.com/fr/{yoox.item[i]}/item').content).select('#compositionInfo > div.font-sans.info-body.padding-half-top'))
    if i%20==0:
        print(i)

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480


In [86]:
for i in range(500,2000):
    if i%5==0:
        t.sleep(3)
    soup=BS(requests.get(f'http://yoox.com/fr/{yoox.item[i]}/item').content)
    yoox.colors[i]=str(soup.select(f'#color{yoox.item[i]} > img'))
    yoox.compositions[i]=str(soup.select('#compositionInfo > div.font-sans.info-body.padding-half-top'))
    if i%100==0:
        print(i)

500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [87]:
for i in range(2000,5000):
    if i%5==0:
        t.sleep(3)
    soup=BS(requests.get(f'http://yoox.com/fr/{yoox.item[i]}/item').content)
    yoox.colors[i]=str(soup.select(f'#color{yoox.item[i]} > img'))
    yoox.compositions[i]=str(soup.select('#compositionInfo > div.font-sans.info-body.padding-half-top'))
    if i%100==0:
        print(i)

2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [91]:
# yoox.colors.str.extract(r'<img alt="(.*)" class="hide"')

,0
0,Camel
1,Bleu
2,Plomb
3,Bleu foncé
4,Beige
5,Bordeaux
6,Beige
7,Sable
8,Blanc
9,Camel


In [92]:
for i in range(5000,10000):
    if i%5==0:
        t.sleep(3)
    soup=BS(requests.get(f'http://yoox.com/fr/{yoox.item[i]}/item').content)
    yoox.colors[i]=str(soup.select(f'#color{yoox.item[i]} > img'))
    yoox.compositions[i]=str(soup.select('#compositionInfo > div.font-sans.info-body.padding-half-top'))
    if i%300==0:
        print(i)

5100
5400
5700
6000
6300
6600
6900
7200
7500
7800
8100
8400
8700
9000
9300
9600
9900


In [93]:
yoox.colors=yoox.colors.str.extract(r'<img alt="(.*)" class="hide"')

In [100]:
yoox.drop(yoox[(yoox.colors=='Bronze') | (yoox.colors=='Transparent') | (yoox.colors=='Cuir')].index,inplace=True)

In [120]:
yoox.drop('categories',axis=1,inplace=True)

In [105]:
yoox.to_csv('yoox_v2.csv')

In [107]:
yoox.categories.value_counts()

0         134956
Blouse       898
Name: categories, dtype: int64

In [108]:
yoox.columns

Index(['brand', 'url', 'price_actual', 'size_30', 'size_32', 'size_34',
       'size_36', 'size_38', 'size_40', 'size_42', 'size_44', 'size_46',
       'size_48', 'discount_rate', '_Blouses', '_Bodycon Dress',
       '_Bomber Jackets', '_Cardigans', '_Coats', '_Culottes', '_Denim Pants',
       '_Down-coat', '_Hoodies', '_Jackets', '_Jumpers', '_Jumpsuits',
       '_Knee Length Skirt', '_Long Coats', '_Maxi Dress', '_Maxi Skirt',
       '_Midi Dress', '_Midi Skirt', '_Mini Dress', '_Mini Skirt',
       '_One piece Swimsuit', '_Pyjamas', '_Shirts', '_Shorts', '_T-shirts',
       '_Tops', '_Trench Coats', '_Trousers', '_Turtle Necks', 'item',
       'colors', 'compositions', 'categories'],
      dtype='object')

In [111]:
yoox[:10000][['_Blouses', '_Bodycon Dress',
       '_Bomber Jackets', '_Cardigans', '_Coats', '_Culottes', '_Denim Pants',
       '_Down-coat', '_Hoodies', '_Jackets', '_Jumpers', '_Jumpsuits',
       '_Knee Length Skirt', '_Long Coats', '_Maxi Dress', '_Maxi Skirt',
       '_Midi Dress', '_Midi Skirt', '_Mini Dress', '_Mini Skirt',
       '_One piece Swimsuit', '_Pyjamas', '_Shirts', '_Shorts', '_T-shirts',
       '_Tops', '_Trench Coats', '_Trousers', '_Turtle Necks']].sum()

_Blouses                898
_Bodycon Dress          285
_Bomber Jackets         282
_Cardigans              218
_Coats                   70
_Culottes                65
_Denim Pants            330
_Down-coat               14
_Hoodies                165
_Jackets                369
_Jumpers                746
_Jumpsuits               91
_Knee Length Skirt      698
_Long Coats              90
_Maxi Dress             194
_Maxi Skirt             124
_Midi Dress             161
_Midi Skirt             403
_Mini Dress             812
_Mini Skirt             139
_One piece Swimsuit       1
_Pyjamas                  2
_Shirts                 683
_Shorts                 161
_T-shirts               475
_Tops                   523
_Trench Coats           166
_Trousers              1710
_Turtle Necks           125
dtype: int64

In [116]:
yoox=yoox.dropna().drop_duplicates(keep='first')

In [121]:
yoox.to_csv('yoox_v3.csv')

In [293]:
yoox=pd.read_csv('yoox_v3.csv').drop('Unnamed: 0',axis=1)

In [294]:
yoox.compositions.value_counts()

[<div class="font-sans info-body padding-half-top">100% Coton</div>]                                                                         1276
[<div class="font-sans info-body padding-half-top">100% Soie</div>]                                                                          1096
[<div class="font-sans info-body padding-half-top">100% Viscose</div>]                                                                        365
[<div class="font-sans info-body padding-half-top">100% Polyester</div>]                                                                      362
[<div class="font-sans info-body padding-half-top">98% Coton, 2% Élasthanne</div>]                                                            209
[<div class="font-sans info-body padding-half-top">100% Laine vierge</div>]                                                                   209
[<div class="font-sans info-body padding-half-top">100% Laine</div>]                                                        

In [295]:
yoox.compositions.str.replace('\[<div class="font-sans info-body padding-half-top">','').str.replace("</div>\]",'').value_counts().shape

(2706,)

In [296]:
yoox.compositions=yoox.compositions.str.replace('\[<div class="font-sans info-body padding-half-top">','').str.replace("</div>\]",'')

In [297]:
yoox.compositions.value_counts().head(15)

100% Coton                    1276
100% Soie                     1096
100% Viscose                   365
100% Polyester                 362
100% Laine vierge              209
98% Coton, 2% Élasthanne       209
100% Laine                     207
100% Polyamide                 119
97% Coton, 3% Élasthanne       113
100% Cachemire                 106
100% Cuir d'agneau              69
69% Acétate, 31% Soie           66
95% Viscose, 5% Élasthanne      55
83% Viscose, 17% Polyester      48
96% Soie, 4% Élasthanne         44
Name: compositions, dtype: int64

In [298]:
yoox.drop(yoox.compositions[yoox.compositions.str.contains('100%')& (yoox.compositions.str.contains(','))].index,inplace=True)

In [299]:
# 100% :

yoox.compositions[yoox.compositions.str.contains('100%')].value_counts()

100% Coton                 1276
100% Soie                  1096
100% Viscose                365
100% Polyester              362
100% Laine vierge           209
100% Laine                  207
100% Polyamide              119
100% Cachemire              106
100% Cuir d'agneau           69
100% Cuir                    36
100% Lin                     20
100% Laine mérinos           19
100% Rayonne                 17
100% Nylon                   13
100% Cuir de veau            12
100% Cupro                   10
100% Polyuréthane             8
100% Peau de chèvre           7
100% Polyéthylène             5
100% Coton biologique         4
100% Ramie                    4
100% Lyocell                  3
100% Modal                    3
100% Acétate                  3
100% Acrylique                2
100% Laine d'alpaga           2
100% Cuir d'ovin              1
100% Super 100s Laine         1
100% Cuir de buffle           1
100% Mouton                   1
100% Viscose EcoVero          1
100% Lai

In [300]:
yoox['new_compositions']=0

In [301]:
yoox['new_compositions'][yoox.compositions.str.contains('100%')&yoox.compositions.str.contains('Laine')|yoox.compositions.str.contains('Mouton')]='Laine'
yoox['new_compositions'][yoox.compositions.str.contains('100%')&yoox.compositions.str.contains('Coton')]='Coton'
yoox['new_compositions'][yoox.compositions.str.contains('100%')&yoox.compositions.str.contains('Soie')]='Soie'
yoox['new_compositions'][yoox.compositions.str.contains('100%')&yoox.compositions.str.contains('Poly')]='Poly...'

yoox['new_compositions'][yoox.compositions.str.contains('100%')&(yoox.compositions.str.contains('Viscose')|yoox.compositions.str.contains('Rayon'))|(yoox.compositions.str.contains('Modal'))]='Viscose'
yoox['new_compositions'][yoox.compositions.str.contains('100%')&yoox.compositions.str.contains('Polyester')]='Polyester'
yoox['new_compositions'][yoox.compositions.str.contains('100%')&yoox.compositions.str.contains('Cachemire')]='Cachemire'
yoox['new_compositions'][yoox.compositions.str.contains('100%')&(yoox.compositions.str.contains('Cuir') | yoox.compositions.str.contains('Peau'))]='Cuir'
yoox['new_compositions'][yoox.compositions.str.contains('100%')&yoox.compositions.str.contains('Lin')]='Lin'
yoox['new_compositions'][yoox.compositions.str.contains('100%')&(yoox.compositions.str.contains('Polyamid')|yoox.compositions.str.contains('Nylon'))]='Polyamide'
yoox['new_compositions'][yoox.compositions.str.contains('100%')&yoox.compositions.str.contains('Fourrure')]='Fourrure'
yoox['new_compositions'][yoox.compositions.str.contains('100%')&yoox.compositions.str.contains('Chanvre')]='Chanvre'
yoox['new_compositions'][yoox.compositions.str.contains('100%')&yoox.compositions.str.contains('Acrylique')]='Acrylique'






In [302]:
yoox=pd.get_dummies(data=yoox,columns=['new_compositions'])

In [303]:
yoox.head()

,brand,url,price_actual,size_30,size_32,size_34,size_36,size_38,size_40,size_42,...,new_compositions_Coton,new_compositions_Cuir,new_compositions_Fourrure,new_compositions_Laine,new_compositions_Lin,new_compositions_Poly...,new_compositions_Polyamide,new_compositions_Polyester,new_compositions_Soie,new_compositions_Viscose
0,8 by YOOX,http://yoox.com/fr/41925191HP/item,249.0,0,0,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
2,MARNI,http://yoox.com/fr/14007863DH/item,218.0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,MARNI,http://yoox.com/fr/35421773RV/item,421.0,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,1
4,8 by YOOX,http://yoox.com/fr/13381785CM/item,119.0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
5,STELLA McCARTNEY,http://yoox.com/fr/41796284SB/item,257.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [304]:
yoox.columns

Index(['brand', 'url', 'price_actual', 'size_30', 'size_32', 'size_34',
       'size_36', 'size_38', 'size_40', 'size_42', 'size_44', 'size_46',
       'size_48', 'discount_rate', '_Blouses', '_Bodycon Dress',
       '_Bomber Jackets', '_Cardigans', '_Coats', '_Culottes', '_Denim Pants',
       '_Down-coat', '_Hoodies', '_Jackets', '_Jumpers', '_Jumpsuits',
       '_Knee Length Skirt', '_Long Coats', '_Maxi Dress', '_Maxi Skirt',
       '_Midi Dress', '_Midi Skirt', '_Mini Dress', '_Mini Skirt',
       '_One piece Swimsuit', '_Pyjamas', '_Shirts', '_Shorts', '_T-shirts',
       '_Tops', '_Trench Coats', '_Trousers', '_Turtle Necks', 'item',
       'colors', 'compositions', 'new_compositions_0',
       'new_compositions_Acrylique', 'new_compositions_Cachemire',
       'new_compositions_Chanvre', 'new_compositions_Coton',
       'new_compositions_Cuir', 'new_compositions_Fourrure',
       'new_compositions_Laine', 'new_compositions_Lin',
       'new_compositions_Poly...', 'new_compositio

In [305]:
yoox.head()

,brand,url,price_actual,size_30,size_32,size_34,size_36,size_38,size_40,size_42,...,new_compositions_Coton,new_compositions_Cuir,new_compositions_Fourrure,new_compositions_Laine,new_compositions_Lin,new_compositions_Poly...,new_compositions_Polyamide,new_compositions_Polyester,new_compositions_Soie,new_compositions_Viscose
0,8 by YOOX,http://yoox.com/fr/41925191HP/item,249.0,0,0,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
2,MARNI,http://yoox.com/fr/14007863DH/item,218.0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,MARNI,http://yoox.com/fr/35421773RV/item,421.0,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,1
4,8 by YOOX,http://yoox.com/fr/13381785CM/item,119.0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
5,STELLA McCARTNEY,http://yoox.com/fr/41796284SB/item,257.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [306]:
import re
yoox.new_compositions_Cachemire[yoox.new_compositions_0==1]=yoox.compositions[yoox['new_compositions_0']==1].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Cachemire',x)))

In [307]:
yoox.new_compositions_Cachemire=yoox.new_compositions_Cachemire.astype('str').str.replace(r'(0.$)','0').astype(float)

In [308]:
yoox.new_compositions_Chanvre[yoox.new_compositions_0==1]=yoox.compositions[yoox['new_compositions_0']==1].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Chanvre',x)))
yoox.new_compositions_Coton[yoox.new_compositions_0==1]=yoox.compositions[yoox['new_compositions_0']==1].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Coton',x)))
yoox.new_compositions_Cuir[yoox.new_compositions_0==1]=yoox.compositions[yoox['new_compositions_0']==1].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Cuir',x)))
yoox.new_compositions_Cuir[yoox.new_compositions_Cuir==0]=yoox.compositions[yoox['new_compositions_Cuir']==0].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Peau',x)))

yoox.new_compositions_Fourrure[yoox.new_compositions_0==1]=yoox.compositions[yoox['new_compositions_0']==1].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Fourrure',x)))
yoox.new_compositions_Laine[yoox.new_compositions_0==1]=yoox.compositions[yoox['new_compositions_0']==1].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Laine',x)))
yoox.new_compositions_Laine[yoox.new_compositions_Laine==0]=yoox.compositions[yoox['new_compositions_Laine']==0].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Mouton',x)))
yoox.new_compositions_Lin[yoox.new_compositions_0==1]=yoox.compositions[yoox['new_compositions_0']==1].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Lin',x)))
yoox.new_compositions_Polyamide[yoox.new_compositions_0==1]=yoox.compositions[yoox['new_compositions_0']==1].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Polyalide',x)))
yoox.new_compositions_Polyamide[yoox.new_compositions_Polyamide==0]=yoox.compositions[yoox['new_compositions_Polyamide']==0].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Nylon',x)))

yoox.new_compositions_Polyester[yoox.new_compositions_0==1]=yoox.compositions[yoox['new_compositions_0']==1].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Polyester',x)))
yoox.new_compositions_Soie[yoox.new_compositions_0==1]=yoox.compositions[yoox['new_compositions_0']==1].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Soie',x)))
yoox.new_compositions_Viscose[yoox.new_compositions_0==1]=yoox.compositions[yoox['new_compositions_0']==1].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Viscose',x)))
yoox.new_compositions_Viscose[yoox.new_compositions_Viscose==0]=yoox.compositions[yoox['new_compositions_Viscose']==0].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Rayon',x)))
yoox.new_compositions_Viscose[yoox.new_compositions_Viscose==0]=yoox.compositions[yoox['new_compositions_Viscose']==0].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Modal',x)))
yoox.new_compositions_Acrylique[yoox.new_compositions_Viscose==0]=yoox.compositions[yoox['new_compositions_Viscose']==0].apply(lambda x: '0.'+''.join(re.findall('(\d+)\d?% Acrylique',x)))

In [309]:
yoox.new_compositions_Chanvre=yoox.new_compositions_Chanvre.astype('str').str.replace(r'(0.$)','0').astype(float)
yoox.new_compositions_Coton=yoox.new_compositions_Coton.astype('str').str.replace(r'(0.$)','0').astype(float)
yoox.new_compositions_Cuir=yoox.new_compositions_Cuir.astype('str').str.replace(r'(0.$)','0').astype(float)
yoox.new_compositions_Fourrure=yoox.new_compositions_Fourrure.astype('str').str.replace(r'(0.$)','0').astype(float)
yoox.new_compositions_Laine=yoox.new_compositions_Laine.astype('str').str.replace(r'(0.$)','0').astype(float)
yoox.new_compositions_Lin=yoox.new_compositions_Lin.astype('str').str.replace(r'(0.$)','0').astype(float)
yoox.new_compositions_Polyamide=yoox.new_compositions_Polyamide.astype('str').str.replace(r'(0.$)','0').astype(float)
yoox.new_compositions_Polyester=yoox.new_compositions_Polyester.astype('str').str.replace(r'(0.$)','0').astype(float)
yoox.new_compositions_Soie=yoox.new_compositions_Soie.astype('str').str.replace(r'(0.$)','0').astype(float)
yoox.new_compositions_Viscose=yoox.new_compositions_Viscose.astype('str').str.replace(r'(0.$)','0').astype(float)
yoox.new_compositions_Acrylique=yoox.new_compositions_Acrylique.astype('str').str.replace(r'(0.$)','0').astype(float)


In [311]:
yoox.drop('new_compositions_0',axis=1,inplace=True)

In [317]:
yoox=pd.get_dummies(data=yoox,columns=['colors'],drop_first=True)

In [321]:
yoox.to_csv('yoox_final.csv')

In [320]:
yoox.dtypes

brand                     object
url                       object
price_actual             float64
size_30                    int64
size_32                    int64
size_34                    int64
size_36                    int64
size_38                    int64
size_40                    int64
size_42                    int64
size_44                    int64
size_46                    int64
size_48                    int64
discount_rate            float64
_Blouses                   int64
_Bodycon Dress             int64
_Bomber Jackets            int64
_Cardigans                 int64
_Coats                     int64
_Culottes                  int64
_Denim Pants               int64
_Down-coat                 int64
_Hoodies                   int64
_Jackets                   int64
_Jumpers                   int64
_Jumpsuits                 int64
_Knee Length Skirt         int64
_Long Coats                int64
_Maxi Dress                int64
_Maxi Skirt                int64
          